- [Run SimKNN v2](#Run-SimKNN-v2)
- [Run SimKNN v1](#Run-SimKNN-v1)

In [1]:
import numpy as np
import pandas as pd
import tool
import simKNN as sk
from simKNNv2 import SimKNN
from khaiii import KhaiiiApi
from tqdm.notebook import tqdm
import pickle

In [2]:
%%time
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
display(song, genre, train, val, test)

CPU times: user 10.3 s, sys: 925 ms, total: 11.3 s
Wall time: 11.3 s


In [4]:
display(train, val, test)

,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[일렉],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡의, 우울, 분위기, 드라이브, 산책]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[기분전환],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름],4,2015-06-06 09:52:01.000


In [24]:
train80 = pd.read_json("arena_data/orig/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20q = pd.read_json("arena_data/questions/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20a = pd.read_json("arena_data/answers/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
result_example = pd.read_json("arena_data/results/results_1.json")
display(train80, val20q, val20a, result_example)

,id,plylst_title,songs,tags,like_cnt,updt_date
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[힐링, 휴식, 밤, 새벽]",12,2016-06-23 10:06:27.000
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝],0,2013-08-15 13:17:11.000
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",[뉴에이지],23,2015-09-03 16:51:50.000
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[하드락, 록스피릿, 댄스]",1,2017-01-09 15:41:25.000
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[힐링, 휴식, 기분전환]",715,2016-02-22 12:32:50.000
...,...,...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[90년생, 회상, 추억, 좋은노래, 80년생, 옛날노래]",155,2020-01-15 15:15:45.000
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝],8,2010-03-23 00:03:00.000
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[여행, 발라드, 기분전환, 사랑]",3,2019-05-15 13:26:07.000
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[소울, 알앤비]",51,2013-12-24 14:40:01.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,[],[],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[274165, 552540, 104677, 188486, 271300, 25864...",[드라이브],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[275414, 477905, 256688, 276685, 109237, 54408...",[랩],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,[],[],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[632115, 307581, 357367, 112832, 648635, 60663...",[],612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[222048, 562040, 295682, 474983, 405990, 61620...",[잔잔한],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[158590, 375777, 598734, 63734, 112399, 76888,...",[힐링],7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,[],[],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[353960, 221785, 234462, 22976, 31784, 403481,...",[],0,2015-05-05 13:29:48.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,"[231820, 13676, 593705, 23746, 429366, 376182,...",[재즈],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[448698, 156706, 185733, 117595, 497066, 28628...",[가을],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[407073, 529965, 700041, 157285, 578509, 65827...",[힙합],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,"[445568, 114172, 234233, 348174, 521628, 43010...",[슬픔],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[468023, 637294, 130632, 340538, 290477, 39356...","[위로, 혼자있고싶을때, 따뜻한]",612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[335474, 105039, 401182, 398584, 311731, 51024...",[까페],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[370002, 551409, 95323, 346969, 579728, 60117,...","[휴식, 잔잔한]",7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,"[572238, 174778, 198528, 280899, 310653, 41008...",[팝],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[315273, 245317, 359517, 593278, 594982, 22588...",[발라드],0,2015-05-05 13:29:48.000


,id,songs,tags
0,119064,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
1,64891,"[144663, 357367, 366786, 654757, 133143, 67511...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 힐링, 사랑, 새벽, 밤, 카페]"
2,68628,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
3,100993,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
4,10417,"[144663, 116573, 366786, 654757, 133143, 67511...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
...,...,...,...
23010,115244,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 드라이브, 힐링, 사랑, 새벽, 밤, 카페]"
23011,124493,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 사랑, 새벽, 밤, 카페]"
23012,65279,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
23013,53828,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"


#### Preprocessing

In [5]:
%%time
lst = [train, val, test]
Xs = []
for data in lst:
    api = KhaiiiApi()
    ko_nouns = {}
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dense=True)
    ko_nouns["title"] = tool.ko_extract_tag_from(ko_title, api, limit=None)
    ko_nouns["tag"] = tool.ko_extract_tag_from(ko_tag, api)
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dtype="list", dense=True)
    new_data = data[['id', 'plylst_title', 'songs']]
    new_data["tags"] = np.nan
    new_data["tags"] = new_data["tags"].astype('object')
    for i in tqdm(data.index):
        a = ko_tag[i] if i in ko_tag else []
        b = en_tag[i] if i in en_tag else []
        c = ko_nouns["title"][i] if i in ko_nouns["title"] else []
        d = ko_nouns["tag"][i] if i in ko_nouns["tag"] else []
        e = list(set(a + b + c + d))
        new_data.at[i, "tags"] = e
    Xs.append(new_data)
display(Xs[0], Xs[1], Xs[2])
with open("bin/Xs.p", 'wb') as f:
    pickle.dump(Xs, f)

/Users/jehunyoo/anaconda3/envs/cosmos/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/jehunyoo/anaconda3/envs/cosmos/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,id,plylst_title,songs,tags
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[음악, 여행, 락]"
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[회상, 요즘, 추억, 말]"
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[잔잔한, 곡, 까페]"
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[캐럴, 겨울노래, 때, 따듯한, 날, 크리스마스, 겨울, 분위기, 눈, 캐, 노래..."
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...","[댄스, 추억, ㅋ, 노래]"
...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[extreme, 밴드, 록메탈, 락메탈, 사운드, 탈, 밴드사운드, 록, 메탈, ..."
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...","[최신, 인기, 모음, 일렉, 리스너]"
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 그리움, 이야기, 눈물, 나의, 사랑, 주인공, 친구, 앞]"
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[버스, 퇴근버스, 퇴근길, 잔잔한, 퇴근, Pop, 풍경, 마무리, 하루]"


,id,plylst_title,songs,tags
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[]
1,131447,앨리스테이블,[],[이블]
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[]
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[]
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[]
...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한]
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[가족, 때, 어머니, 위로받고싶을때, 힘들때, 위, 아빠]"
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[]
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[]


,id,plylst_title,songs,tags
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[]
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[티, 인디아티스트들의추천음악, 음악, 나만의, Best, 의추천, 트]"
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브]
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기]
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[]
...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억]
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡, 산책, 우울, 분위기, 드라이브, 띵곡의]"
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...","[기분전환, 전환, 기분]"
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름]


CPU times: user 1min 41s, sys: 914 ms, total: 1min 42s
Wall time: 1min 42s


In [6]:
with open("bin/Xs.p", 'rb') as f:
    Xs = pickle.load(f)
display(Xs[0], Xs[1], Xs[2])

,id,plylst_title,songs,tags
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[음악, 여행, 락]"
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[회상, 요즘, 추억, 말]"
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[잔잔한, 곡, 까페]"
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[캐럴, 겨울노래, 때, 따듯한, 날, 크리스마스, 겨울, 분위기, 눈, 캐, 노래..."
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...","[댄스, 추억, ㅋ, 노래]"
...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[extreme, 밴드, 록메탈, 락메탈, 사운드, 탈, 밴드사운드, 록, 메탈, ..."
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...","[최신, 인기, 모음, 일렉, 리스너]"
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 그리움, 이야기, 눈물, 나의, 사랑, 주인공, 친구, 앞]"
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[버스, 퇴근버스, 퇴근길, 잔잔한, 퇴근, Pop, 풍경, 마무리, 하루]"


,id,plylst_title,songs,tags
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[]
1,131447,앨리스테이블,[],[이블]
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[]
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[]
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[]
...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한]
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[가족, 때, 어머니, 위로받고싶을때, 힘들때, 위, 아빠]"
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[]
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[]


,id,plylst_title,songs,tags
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[]
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[티, 인디아티스트들의추천음악, 음악, 나만의, Best, 의추천, 트]"
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브]
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기]
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[]
...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억]
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡, 산책, 우울, 분위기, 드라이브, 띵곡의]"
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...","[기분전환, 전환, 기분]"
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름]


In [14]:
data, X, v = train[["id", "songs"]], test[["id", "songs"]], val[["id", "songs"]]
display(data, X, v)

,id,songs
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,..."
...,...,...
115066,120325,"[429629, 441511, 612106, 516359, 691768, 38714..."
115067,106976,"[321330, 216057, 534472, 240306, 331098, 23288..."
115068,11343,"[50512, 249024, 250608, 371171, 229942, 694943..."
115069,131982,"[533534, 608114, 343608, 417140, 609009, 30217..."


,id,songs
0,70107,"[398985, 449403, 411543, 528044, 143048, 98020..."
1,7461,"[196298, 269984, 267805, 175867, 529244, 63825..."
2,90348,"[273433, 331003, 68432, 411659, 117793, 616860..."
3,58617,"[702227, 48152, 440008, 358488, 701041, 540721..."
4,102395,"[630683, 481582, 528550, 285114, 506667, 17922..."
...,...,...
10735,137930,"[323755, 397594, 445908, 570242, 221853, 20018..."
10736,936,"[105140, 582252, 199262, 422915, 547967, 48791..."
10737,110589,"[21976, 207746, 40025, 31635, 567462, 641799, ..."
10738,2605,"[234554, 265033, 507260, 83092, 366757, 497097..."


,id,songs
0,118598,"[373313, 151080, 275346, 696876, 165237, 52593..."
1,131447,[]
2,51464,"[529437, 516103, 360067, 705713, 226062, 37089..."
3,45144,"[589668, 21711, 570151, 320043, 13930, 599327,..."
4,79929,"[672718, 121924, 102694, 683657, 201558, 38511..."
...,...,...
23010,101722,"[75842, 26083, 244183, 684715, 500593, 508608,..."
23011,122127,"[450275, 487671, 561031, 663944, 628672, 59121..."
23012,77438,"[625875, 464051, 11657, 236393, 358186, 213435..."
23013,36231,"[161094, 665833, 688145, 432735, 439938, 12665..."


In [13]:
val[val.id == 490]

,id,plylst_title,songs,tags,like_cnt,updt_date
6956,490,,"[381653, 566122, 158539, 261549, 368705]",[겨울],0,2019-11-19 13:23:31.000


In [4]:
# FIXME
def pred_info(u, track_ids, relevances, song=song, test=test):
    pd.set_option('display.max_rows', None)
    plylst_id = test.id[u]
    _track_ids = test.songs[u]
    print(plylst_id)
    pred = pd.DataFrame(song.song_name[track_ids], columns=["id", "song_name"])
    original = pd.DataFrame(song.song_name[_track_ids], columns=["id", "song_name"])
    display(pd.concat([pred, original], axis=1, ignore_index=True))

### Run SimKNN v2

In [4]:
%%time
with open("bin/Xs.p", 'rb') as f:
        Xs = pickle.load(f)
simknn = SimKNN(k=200, sim=["cosine", "idf", "cosine"], beta=0.5, gamma=0.5)
simknn.fit(x=Xs[0])
start, end = None, None
pred1 = simknn.predict(X=Xs[1], start=start, end=end)
pred2 = simknn.predict(X=Xs[1], start=start, end=end, inter=False)
# print(pred1.loc[[i for i in range(start, end)], ["songs", "tags", "rel_tags"]])
# print(pred2.loc[[i for i in range(start, end)], ["songs", "tags", "rel_tags"]])

19(tags) 0
19 : too many indices for array

                                                songs  \
19  [551411, 177460, 140867, 663256, 217365, 58179...   
20  [554751, 11657, 140867, 376435, 146989, 701801...   
21  [253755, 107220, 173943, 90557, 339802, 67655,...   

                                          tags  \
19                                         NaN   
20  [장르, 음악, 다양한, 기분, 전환, 기분전환, 노래, 락, 모음, 감성]   
21     [발라드, 팝, 락, 인디, 댄스, 봄, 여름, 겨울, JPOP, 비]   

                                             rel_tags  
19                                                NaN  
20  [0.2028133999937481, 0.19473787515314844, 0.15...  
21  [0.31510330260716646, 0.08366353636607803, 0.0...  


/Users/jehunyoo/AI/kakao/MelonPlaylistContinuation/simKNNv2.py:99: RuntimeWarning: invalid value encountered in double_scalars
  for track in tracks])


19(tags) 0
19 : too many indices for array

                                                songs  \
19  [707948, 234921, 235540, 235533, 235495, 23542...   
20  [564439, 323077, 12392, 73134, 584843, 204900,...   
21  [219025, 440160, 357367, 174749, 373037, 19323...   

                                          tags  \
19                                         NaN   
20  [장르, 음악, 다양한, 기분, 전환, 기분전환, 노래, 락, 모음, 감성]   
21      [발라드, 팝, 락, 인디, 댄스, 겨울, 여름, 봄, 클럽, 가을]   

                                             rel_tags  
19                                                NaN  
20  [0.20656956228811033, 0.19292935338542774, 0.1...  
21  [0.3644347255514558, 0.08864628459359734, 0.06...  
CPU times: user 1min 32s, sys: 438 ms, total: 1min 32s
Wall time: 1min 33s


In [8]:
p1 = pred1.loc[[i for i in range(start, end)], ["songs", "tags", "rel_tags"]]
p2 = pred2.loc[[i for i in range(start, end)], ["songs", "tags", "rel_tags"]]
display(p1, p2)

,songs,tags,rel_tags
19,"[551411, 177460, 140867, 663256, 217365, 58179...",NaN,NaN
20,"[554751, 11657, 140867, 376435, 146989, 701801...","[장르, 음악, 다양한, 기분, 전환, 기분전환, 노래, 락, 모음, 감성]","[0.2028133999937481, 0.19473787515314844, 0.15..."
21,"[253755, 107220, 173943, 90557, 339802, 67655,...","[발라드, 팝, 락, 인디, 댄스, 봄, 여름, 겨울, JPOP, 비]","[0.31510330260716646, 0.08366353636607803, 0.0..."


,songs,tags,rel_tags
19,"[707948, 234921, 235540, 235533, 235495, 23542...",NaN,NaN
20,"[564439, 323077, 12392, 73134, 584843, 204900,...","[장르, 음악, 다양한, 기분, 전환, 기분전환, 노래, 락, 모음, 감성]","[0.20656956228811033, 0.19292935338542774, 0.1..."
21,"[219025, 440160, 357367, 174749, 373037, 19323...","[발라드, 팝, 락, 인디, 댄스, 겨울, 여름, 봄, 클럽, 가을]","[0.3644347255514558, 0.08864628459359734, 0.06..."


In [41]:
p1.songs[19]

array([551411, 177460, 140867, 663256, 217365, 581799, 360825,  28832,
       140638, 207558, 495557,  24087, 470684, 153807,  21125, 465739,
       157861, 534299, 464051, 686809, 243951, 612944, 100733, 476680,
        41328,  16293, 314302, 562575,  43627, 554106, 140192,  72222,
        53061,  76084, 166761, 580607, 513419, 639289,  32120, 402769,
        62200, 333595, 457943, 482423, 403082, 679436, 101480, 705098,
       280915, 502921, 284554, 325306, 474801, 696317, 263126, 470129,
       274504, 572238, 388258, 258957, 573694, 585388, 364319, 476458,
       205757, 574944, 484934,  41105, 425904,  72797, 380626, 649290,
       165793, 175247, 505710, 236393,  52237, 542127, 555344, 681345,
       557133, 317882, 563111, 310907,   2748, 330218, 359929, 142969,
       477991, 480973,  28266, 625875, 623110, 704380, 260991, 314714,
       534989, 105272, 381746, 660013])

In [44]:
song.song_name[train[train.id == 133661].songs.to_numpy()[0]]

256899                                         Martini Blue
348948               입씨름 (Prod. By Cosmicboy) (Feat. Sik-K)
447455                  WHY DO FUCKBOIS HANG OUT ON THE NET
302892                                앵무새 (Feat. 기리보이, 한요한)
527223          평화 (Prod. By Cosmic Boy) (Feat. Crush, 최엘비)
440696                        재미로 (Feat. Futuristic Swaver)
411368                                 flex (Prod. By 기리보이)
48454                                 Vacation (Feat. SOLE)
228303          옛날거 (Prod. By 기리보이, MISU) (Feat. Kid Milli)
146819                                키보드 (Prod. By Hansen)
4346                                             X발 (XIBAL)
636800                                          How to Live
196447                                        Shotta Flow 3
638149                      파티피플 (Feat. 염따, Uneducated Kid)
55620     잔돈은 됐어요 (Keep The Change) (Feat. Garie Of Lees...
591047                                     불꽃놀이 (Fireworks)
2242                                    

In [28]:
display(song.song_name[p1.songs[19]], song.song_name[p2.songs[19]])

551411                      One Dance (Feat. WizKid & Kyla)
177460                          Slow Motion (Album Version)
140867                                     Someone Like You
663256    Call You Mine (Feat. Geologic Of The Blue Scho...
217365                       Don`t Let Me Down (Feat. Daya)
                                ...                        
314714                              I Knew You Were Trouble
534989                                   Castle on the Hill
105272                                 Locked Out Of Heaven
381746                            Cracks of My Broken Heart
660013                                      Want To Want Me
Name: song_name, Length: 100, dtype: object

707948                    보란듯이 (Feat.정슬기)
234921                                짝사랑
235540                               Dear
235533                       Walk Me Home
235495    Burn It To The Ground (Gh Ver.)
                       ...               
242052                             달려라 하니
242124                          주를 위한 이곳에
242180                   Aubrey (Lp Ver.)
242198            이것만은 알고 가 (Before U Go)
242232                Dancing In The Rain
Name: song_name, Length: 100, dtype: object

### Run SimKNN v1

In [10]:
simknn = sk.SimKNN(k=100, metric="idf")
simknn.fit(Xs[0][['id', 'songs']])
pred = simknn.predict(Xs[1][['id', 'songs']], limit=1)
pred

/Users/jehunyoo/AI/kakao/MelonPlaylistContinuation/simKNN.py:72: RuntimeWarning: invalid value encountered in long_scalars
  r_u = np.array([( track, np.sum([S[v] if track in self.data[v] else 0 for v in top]) / norm) for track in tracks])


[(0,
  array([707786, 231688, 232046, 232068, 232172, 232305, 232606, 232874,
         233041, 233239, 233546, 233641, 233969, 234047, 234104, 234105,
         234409, 232030, 231619, 234921, 231601, 229578, 229683, 229711,
         230100, 230131, 230290, 230393, 231018, 231154, 231170, 231174,
         231175, 231306, 231481, 231577, 234462, 235427, 240291, 238725,
         238882, 238927, 239059, 239162, 239479, 239518, 239594, 239673,
         239835, 240023, 240192, 240208, 240235, 240250, 240268, 238794,
         238699, 235495, 238688, 235533, 235540, 236028, 236224, 236261,
         236332, 236393, 236639, 237111, 237464, 238062, 238067, 238468,
         238576, 238621, 229380, 229268, 229168, 222253, 222350, 222451,
         222464, 222481, 222920, 222987, 223071, 223083, 223102, 223159,
         223654, 223673, 223889, 223981, 224139, 222305, 222141, 229121,
         222013, 219528, 219635, 219955]),
  array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
  

In [7]:
simknn2 = SimKNN(k=100)
simknn2.fit(data)
pred2 = simknn2.predict(X, limit=1)
print(pred2)


[(0, array([374617, 688951, 305064, 566257,  45581, 635759, 249594, 524287,
        80103, 227169, 228501, 579592, 275414, 544625, 477609, 163560,
       387621, 272924,  24221, 254896, 331003,  15958, 307012, 232305,
       318871, 609247, 695910, 472395, 297010, 631627, 225426, 144826,
       465471, 513270, 232224, 474175, 157389, 395438, 489157, 622533,
       187991, 534493, 329634, 541219, 602629, 609622, 138031, 388635,
       628459, 120626, 293024, 115765,   7749, 477905, 371829, 101388,
       656512, 445084, 553894,   8763, 543751, 637303, 234958,  37235,
         1546, 311834, 185513, 311240, 555134, 130275, 679405, 109190,
        15846, 632755, 407044, 284696, 644455, 425758, 618150, 388487,
       413823, 274716,   7837, 303268, 254801, 405542, 155016, 407073,
        54408, 481045,  89571, 386822, 302604, 148788, 221986, 703596,
       181369, 264701, 237407, 486768]), array([0.25800223, 0.23646837, 0.21265428, 0.19829843, 0.19353048,
       0.17944714, 0.16766213, 0.1

In [21]:
u, t1, rel1 = pred[0]
u2, t2, rel2 = pred2[0]
_track_ids = test.songs[u]

In [25]:
common = np.intersect1d(t1, t2)
common

array([ 45581,  54408,  89571, 144826, 157389, 163560, 187991, 225426,
       227169, 228501, 234958, 254801, 254896, 272924, 275414, 297010,
       305064, 307012, 311240, 331003, 374617, 387621, 388635, 395438,
       407044, 407073, 472395, 474175, 477609, 477905, 489157, 513270,
       524287, 534493, 544625, 555134, 609622, 618150, 622533, 628459,
       631627, 632755, 635759, 656512, 688951, 695910])

In [34]:
c1 = [np.where(t1 == i)[0][0] for i in common]
c2 = [np.where(t2 == i)[0][0] for i in common]
print("from cosine to idf")
for i, v in enumerate(common):
    if c2[i] > c1[i]:
        change = "up"
    elif c2[i] < c1[i]:
        change = "down"
    else:
        change = "same"
    print("{}: {} -> {} // {} // diff={}".format(v, c2[i], c1[i], change, c2[i] - c1[i]))

from cosine to idf
45581: 4 -> 2 // up // diff=2
54408: 88 -> 50 // up // diff=38
89571: 90 -> 29 // up // diff=61
144826: 31 -> 84 // down // diff=-53
157389: 36 -> 9 // up // diff=27
163560: 15 -> 44 // down // diff=-29
187991: 40 -> 21 // up // diff=19
225426: 30 -> 52 // down // diff=-22
227169: 9 -> 0 // up // diff=9
228501: 10 -> 34 // down // diff=-24
234958: 62 -> 25 // up // diff=37
254801: 84 -> 64 // up // diff=20
254896: 19 -> 6 // up // diff=13
272924: 17 -> 13 // up // diff=4
275414: 12 -> 14 // down // diff=-2
297010: 28 -> 97 // down // diff=-69
305064: 2 -> 1 // up // diff=1
307012: 22 -> 87 // down // diff=-65
311240: 67 -> 32 // up // diff=35
331003: 20 -> 7 // up // diff=13
374617: 0 -> 38 // down // diff=-38
387621: 16 -> 4 // up // diff=12
388635: 47 -> 43 // up // diff=4
395438: 37 -> 19 // up // diff=18
407044: 74 -> 91 // down // diff=-17
407073: 87 -> 47 // up // diff=40
472395: 27 -> 5 // up // diff=22
474175: 35 -> 16 // up // diff=19
477609: 14 -> 15 // dow

In [43]:
for t in t1:
    print(song[song.id == t].song_gn_gnr_basket.values)

[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0200'])]
[list(['GN2500', 'GN0200'])]
[list(['GN1500'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0200'])]
[list(['GN0200'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0200'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0100'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0500', 'GN0300'])]
[list(['GN0100'])]
[list(['GN0300'])]
[list(['GN1900'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0400', 'GN2500'])]
[list(['GN2500', 'GN0200'])]
[list(['GN2500', 'GN0100'])]
[list(['GN1200'])]
[list(['GN2500', 'GN0100'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0100'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0900'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0500', 'GN0300'])]
[list(['GN0300'])]
[list(['GN2600'])]
[list(['GN0100'])]
[list(['GN0400'])]
[list(['GN0500', 'GN0800'])]
[list(['GN250

In [7]:
# pd.set_option('display.max_rows', None)
u, track_ids, relevances = pred[0]
_track_ids = test.songs[u]
print(song.song_name[track_ids])
print(song.song_name[_track_ids])

227169                                          이럴 줄 알았어
305064                                 Solo (Feat. Alex)
45581                                                답답해
695910                                            내가 아니야
387621                                      I Don`t Care
472395                                              미운사람
254896                                          Hate You
331003                                 사랑과 전쟁 (Narr. 하하)
628459                                   있잖아 (Rock Ver.)
157389                                              미치겠어
688951              ? (물음표) (Feat. 최자 Of 다이나믹듀오, Zion.T)
513270                                               Boo
524287                     Love (Feat. Bumkey, Paloalto)
272924                            Gone (Feat. 주희 Of 에이트)
275414                            Officially Missing You
477609                                             사고쳤어요
474175                                            Baby U
534493          시간이 날 기다려 (Tik 

In [ ]:
%%time
u, track_ids, relevances = next(pred)
# pred_info(u, track_ids, relevances)
_track_ids = test.songs[u]
song.song_name[track_ids], song.song_name[_track_ids]
# display 해주는 함수 만들기
# 겹치는 노래 제외해야 함
# html

In [12]:
size = int(data.shape[0] * (1 - 0.3))
x = data.iloc[size:, :]
x.songs[:]

[112418, 363240, 75991, 386495, 22061]

In [10]:
X_val = val[['id', 'songs']]
model = SimKNN(k=100, rho=0.4, metric="idf")
model.fit(data)
pred = model.predict(X_val, limit=1)
print(pred)


[(0, array([207912, 623047, 422438, 703323, 638488, 322215, 157900, 690767,
        39436, 516376,  78983, 140837, 439161, 472374, 569715, 328908,
       557956,  66475, 262430, 413459, 209622, 217622, 411438, 367826,
       292859, 551157, 187531, 607459, 412246, 394031, 447762, 448547,
       654428, 455806, 646988, 307938, 494552, 545816, 421833, 249378,
       680970, 620311, 616144, 693988, 467269, 585728, 438857, 554509,
       584108, 219415, 259770, 212592, 365613, 606063, 225652, 671973,
       404536, 491808, 382962, 412305, 562903,  89503, 546713, 416417,
       226991, 271335, 693580, 385871, 396028, 381851,  10690, 258135,
       389960, 245272,  21512, 586389, 574308, 359986, 125214, 520034,
       611201, 107413, 268669, 370618, 352807, 587469, 380092, 340089,
       242188, 648022, 379162,  76044, 425357, 283939, 637900, 673758,
       375776, 343420, 407255,  78337]), array([0.67384228, 0.5642619 , 0.42392888, 0.42308025, 0.401394  ,
       0.37508225, 0.31926218, 0.3